In [2]:
import cloudknot as ck
import itertools
ck.set_region('us-west-2')

In [12]:
def afq_cam_can(subject, n_seeds, odf_model, reg_subject):
    import AFQ.data as afqd
    import AFQ.api as api
    import AFQ.mask as afm
    import os.path as op
    import s3fs
    import packaging
    import cython

    local_bids_folder = "cam_can"
    bucket = "cam-can-mri"
    s3_prefix = ""
    s3_prefix_qsiprep = "derivatives/qsiprep"
    scope = 'qsiprep'

    # select subjects from qsiprep
    study_qsiprep = afqd.S3BIDSStudy(
        "Cam-CAN-qsiprep",
        bucket,
        s3_prefix_qsiprep,
        subjects=[subject],
        anon=False,
        random_seed=42)
    sub_name = study_qsiprep.subjects[0].subject_id

    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=sub_name,
        anon=False,
        random_seed=42)
    study.download(
        local_bids_folder,
        include_derivs=scope)
    
    tracking_params = {
        "n_seeds": n_seeds,
        "directions": "prob",
        "odf_model": odf_model,
        "seed_mask": afm.MaskFile(
            "probseg",
            {"label": "WM",
            "space": None}), 
        "seed_threshold": 0.1,
        "stop_mask": afm.ScalarMask("dki_fa"),
        "stop_threshold": 0.2,
    }
    
    reg_template = {
        "b0": "mni_t1",
        "power_map": "mni_t2",
        "dti_fa_subject": "dti_fa_template",
        "subject_sls": "hcp_atlas",
    }
    
    myafq = api.AFQ(
        local_bids_folder,
        dmriprep='all',
        brain_mask=afm.MaskFile(
            "mask",
            {"desc": "brain",
            "space": None}),
        reg_subject=reg_subject,
        reg_template=reg_template[reg_subject],
        scalars=['dki_md', 'dki_fa'],
        tracking_params=tracking_params,
        virtual_frame_buffer=True)
    myafq.export_all()
    
    output_suffix = "nseeds_" + "_".join([str(n_seeds), odf_model, reg_subject]).lower()
    myafq.upload_to_s3(s3fs.S3FileSystem(), op.join(bucket, s3_prefix, "derivatives/afq_" + output_suffix))

In [14]:
afq_knot = ck.Knot(name='afq_cam_can-arh-20200904-1',
                   func=afq_cam_can,
                   base_image='libglxvfb:1',
                   image_github_installs="https://github.com/36000/pyAFQ.git@avoid_dipy_load_tractogram_bug#egg=pyAFQ[fury]",
                   pars_policies=('AmazonS3FullAccess',),
                   bid_percentage=100,
                   max_vcpus=512,
                   memory=64000)

In [15]:
subjects = ["sub-CC120347", "sub-CC121685", "sub-CC210148", "sub-CC210422"]
n_seeds = [1, 2]
odf_model = ["CSD"] #, "DKI"]
reg_subject = ["b0", "dti_fa_subject", "power_map", "subject_sls"]

args = list(itertools.product(subjects, n_seeds, odf_model, reg_subject))
args

[('sub-CC120347', 1, 'CSD', 'b0'),
 ('sub-CC120347', 1, 'CSD', 'dti_fa_subject'),
 ('sub-CC120347', 1, 'CSD', 'power_map'),
 ('sub-CC120347', 1, 'CSD', 'subject_sls'),
 ('sub-CC120347', 2, 'CSD', 'b0'),
 ('sub-CC120347', 2, 'CSD', 'dti_fa_subject'),
 ('sub-CC120347', 2, 'CSD', 'power_map'),
 ('sub-CC120347', 2, 'CSD', 'subject_sls'),
 ('sub-CC121685', 1, 'CSD', 'b0'),
 ('sub-CC121685', 1, 'CSD', 'dti_fa_subject'),
 ('sub-CC121685', 1, 'CSD', 'power_map'),
 ('sub-CC121685', 1, 'CSD', 'subject_sls'),
 ('sub-CC121685', 2, 'CSD', 'b0'),
 ('sub-CC121685', 2, 'CSD', 'dti_fa_subject'),
 ('sub-CC121685', 2, 'CSD', 'power_map'),
 ('sub-CC121685', 2, 'CSD', 'subject_sls'),
 ('sub-CC210148', 1, 'CSD', 'b0'),
 ('sub-CC210148', 1, 'CSD', 'dti_fa_subject'),
 ('sub-CC210148', 1, 'CSD', 'power_map'),
 ('sub-CC210148', 1, 'CSD', 'subject_sls'),
 ('sub-CC210148', 2, 'CSD', 'b0'),
 ('sub-CC210148', 2, 'CSD', 'dti_fa_subject'),
 ('sub-CC210148', 2, 'CSD', 'power_map'),
 ('sub-CC210148', 2, 'CSD', 'subject

In [16]:
ft = afq_knot.map(args, starmap=True)

In [18]:
ck.set_region('us-west-2')
afq_knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
0ab5cfec-96eb-4cbf-9f90-39f65a58c32c        afq-cam-can-arh-20200904-1-0        SUCCEEDED
